In [1]:
!pip install geocoder

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


print("Libraries imported.")

Libraries imported.


In [3]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_of_Lagos").text

In [4]:
soup = BeautifulSoup(data, 'html.parser')

In [5]:
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Category:Neighborhoods of Lagos - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XkjvTwpAICIAAFj1SwIAAACS","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Neighborhoods_of_Lagos","wgTitle":"Neighborhoods of Lagos","wgCurRevisionId":833825338,"wgRevisionId":833825338,"wgArticleId":29517354,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Commons category link is on 

In [6]:
neighborhoodList = []

In [7]:
for row in soup.find_all("div", class_="mw-category")[0].findAll('li'):
    neighborhoodList.append(row.text)

In [8]:
df = pd.DataFrame({"Neighborhood": neighborhoodList})

df.head(20)

,Neighborhood
0,Abule Egba
1,Agege
2,Alimosho
3,Apapa
4,Bariga
5,Iddo Island
6,"Ijora, Lagos"
7,Ikoyi
8,Lekki
9,Obalende


In [9]:
df.shape

(14, 1)

In [10]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Lagos, Nigeria'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [12]:
coords

[[6.647500000000036, 3.303060000000073],
 [6.6256100000000515, 3.312620000000038],
 [6.609270000000038, 3.255800000000022],
 [6.437950000000058, 3.3643600000000333],
 [6.535000000000025, 3.3947200000000635],
 [6.471940000000075, 3.3791700000000446],
 [6.4538853537020975, 3.3663172074042054],
 [6.456060000000036, 3.4422200000000203],
 [6.4094800000000305, 4.0915200000000596],
 [6.4469400000000405, 3.4152800000000525],
 [6.569470000000024, 3.3952000000000453],
 [6.537850000000049, 3.385340000000042],
 [6.465750000000071, 3.544340000000034],
 [6.454700000000059, 3.3887600000000475]]

In [13]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [14]:
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [15]:
df

,Neighborhood,Latitude,Longitude
0,Abule Egba,6.647500,3.303060
1,Agege,6.625610,3.312620
2,Alimosho,6.609270,3.255800
3,Apapa,6.437950,3.364360
4,Bariga,6.535000,3.394720
5,Iddo Island,6.471940,3.379170
6,"Ijora, Lagos",6.453885,3.366317
7,Ikoyi,6.456060,3.442220
8,Lekki,6.409480,4.091520
9,Obalende,6.446940,3.415280


In [16]:
df.to_csv('Neighbhorhood.csv')

In [17]:
address = 'Lagos, Nigeria'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lagos, Nigeria {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lagos, Nigeria 6.4550575, 3.3941795.


In [18]:
map_df = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_df)  
    
map_df

In [19]:
map_df.save('map_of_Lagos.html')

In [20]:
CLIENT_ID = 'UKICY4ZUOFGVCVLALDRPVKQCQ52N2OLFPDKMYPZJ5KLMF11P' # your Foursquare ID
CLIENT_SECRET = '2HX42O0DROOYI11FPJT0RDDNMDZQZ3A4K5D4GFMABHHUYJNI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UKICY4ZUOFGVCVLALDRPVKQCQ52N2OLFPDKMYPZJ5KLMF11P
CLIENT_SECRET:2HX42O0DROOYI11FPJT0RDDNMDZQZ3A4K5D4GFMABHHUYJNI


In [21]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [22]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(229, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abule Egba,6.6475,3.30306,Kotongora Market,6.648938,3.305505,Flea Market
1,Abule Egba,6.6475,3.30306,Mr biggs abule egba,6.649176,3.305269,Burger Joint
2,Abule Egba,6.6475,3.30306,otunba,6.648647,3.306266,Hookah Bar
3,Abule Egba,6.6475,3.30306,"Isokoko Market, Agege",6.658001,3.300299,Farmers Market
4,Abule Egba,6.6475,3.30306,Fagba Pipeline,6.655874,3.317279,Light Rail Station


In [23]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Abule Egba,5,5,5,5,5,5
Agege,8,8,8,8,8,8
Alimosho,4,4,4,4,4,4
Apapa,6,6,6,6,6,6
Bariga,9,9,9,9,9,9
Iddo Island,15,15,15,15,15,15
"Ijora, Lagos",9,9,9,9,9,9
Ikoyi,28,28,28,28,28,28
Obalende,84,84,84,84,84,84


In [24]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))


There are 85 uniques categories.


In [25]:
venues_df['VenueCategory'].unique()[:50]

array(['Flea Market', 'Burger Joint', 'Hookah Bar', 'Farmers Market',
       'Light Rail Station', 'Fast Food Restaurant', 'Campground',
       'Bus Station', 'Market', 'Gym', 'Bus Stop', 'Bar', 'Airport',
       'Hotel', 'Shopping Mall', 'Playground', 'Pizza Place',
       'Indian Restaurant', 'Department Store', 'Boat or Ferry',
       'Convenience Store', 'Restaurant', 'River', 'Pharmacy',
       'Art Gallery', 'Train Station', 'Liquor Store',
       'Rental Car Location', 'Noodle House', 'Fish Market', 'Plaza',
       'Lounge', 'Neighborhood', 'Diner', 'Athletics & Sports',
       'American Restaurant', 'Gym / Fitness Center', 'Golf Course',
       'Sports Bar', 'Wine Bar', 'Café', 'Bistro', 'Ice Cream Shop',
       'Spa', 'African Restaurant', 'Park', 'Pool Hall', 'Arcade',
       'Nightclub', 'Music Venue'], dtype=object)

In [26]:
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [27]:
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(229, 86)


,Neighborhoods,African Restaurant,Airport,American Restaurant,Arcade,Art Gallery,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,Bar,Baseball Stadium,Beer Garden,Bistro,Boat or Ferry,Brazilian Restaurant,Breakfast Spot,Burger Joint,Bus Station,Bus Stop,Café,Campground,Chinese Restaurant,Cocktail Bar,Convenience Store,Convention Center,Department Store,Dessert Shop,Diner,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hookah Bar,Hotel,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Modern European Restaurant,Music Venue,Neighborhood,Nightclub,Noodle House,Park,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool Hall,Recreation Center,Rental Car Location,Resort,Restaurant,River,Shopping Mall,Snack Place,Soccer Field,Spa,Sports Bar,Stables,Stadium,Supermarket,Sushi Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop
0,Abule Egba,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Abule Egba,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Abule Egba,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Abule Egba,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Abule Egba,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(13, 86)


,Neighborhoods,African Restaurant,Airport,American Restaurant,Arcade,Art Gallery,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,Bar,Baseball Stadium,Beer Garden,Bistro,Boat or Ferry,Brazilian Restaurant,Breakfast Spot,Burger Joint,Bus Station,Bus Stop,Café,Campground,Chinese Restaurant,Cocktail Bar,Convenience Store,Convention Center,Department Store,Dessert Shop,Diner,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hookah Bar,Hotel,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Modern European Restaurant,Music Venue,Neighborhood,Nightclub,Noodle House,Park,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool Hall,Recreation Center,Rental Car Location,Resort,Restaurant,River,Shopping Mall,Snack Place,Soccer Field,Spa,Sports Bar,Stables,Stadium,Supermarket,Sushi Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop
0,Abule Egba,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Agege,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.125000,0.000000,0.125,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.125000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Alimosho,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.250000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Apapa,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.000000,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.166667,0.000000,0.000000,0.00000

In [29]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])


5

In [30]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]


In [31]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Abule Egba,0.000000
1,Agege,0.000000
2,Alimosho,0.000000
3,Apapa,0.333333
4,Bariga,0.000000


In [32]:
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 1, 2, 2, 1, 2, 0, 2], dtype=int32)

In [33]:
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [34]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Abule Egba,0.000000,2
1,Agege,0.000000,2
2,Alimosho,0.000000,2
3,Apapa,0.333333,1
4,Bariga,0.000000,2


In [36]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(13, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Abule Egba,0.000000,2,6.64750,3.30306
1,Agege,0.000000,2,6.62561,3.31262
2,Alimosho,0.000000,2,6.60927,3.25580
3,Apapa,0.333333,1,6.43795,3.36436
4,Bariga,0.000000,2,6.53500,3.39472


In [37]:
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(13, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
8,Obalende,0.059524,0,6.446940,3.415280
11,Victoria Garden City,0.117647,0,6.465750,3.544340
12,"Yaba, Lagos",0.083333,0,6.454700,3.388760
3,Apapa,0.333333,1,6.437950,3.364360
6,"Ijora, Lagos",0.222222,1,6.453885,3.366317
0,Abule Egba,0.000000,2,6.647500,3.303060
1,Agege,0.000000,2,6.625610,3.312620
2,Alimosho,0.000000,2,6.609270,3.255800
4,Bariga,0.000000,2,6.535000,3.394720
5,Iddo Island,0.000000,2,6.471940,3.379170


In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [39]:
map_clusters.save('map_clusters.html')


In [40]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
8,Obalende,0.059524,0,6.44694,3.41528
11,Victoria Garden City,0.117647,0,6.46575,3.54434
12,"Yaba, Lagos",0.083333,0,6.45470,3.38876


In [41]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
3,Apapa,0.333333,1,6.437950,3.364360
6,"Ijora, Lagos",0.222222,1,6.453885,3.366317


In [42]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Abule Egba,0.0,2,6.64750,3.30306
1,Agege,0.0,2,6.62561,3.31262
2,Alimosho,0.0,2,6.60927,3.25580
4,Bariga,0.0,2,6.53500,3.39472
5,Iddo Island,0.0,2,6.47194,3.37917
7,Ikoyi,0.0,2,6.45606,3.44222
9,Ogudu,0.0,2,6.56947,3.39520
10,Somolu,0.0,2,6.53785,3.38534
